In [9]:
## Trade data transformation script
import os
import re
import glob
import pandas as pd

# === SETTINGS ===
DATA_DIR = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Trade Data/UK"
FILE_PATTERN = "UK*.xlsx"

# Map output column names to the Trading Partner names in the files
partner_map = {
    "EM": "China",
    "JP": "Japan",
    "EU": "European Union",
    "CH": "Switzerland",
    "AU": "Australia",
    "US": "United States",
}

# === PROCESSING ===
rows = []

# Glob all matching files
file_paths = glob.glob(os.path.join(DATA_DIR, FILE_PATTERN))

for path in file_paths:
    filename = os.path.basename(path)

    # Extract year from filename (EU2025 → 2025)
    m = re.search(r"UK(\d{4})", filename)
    if not m:
        continue
    year = int(m.group(1))

    df = pd.read_excel(path)

    # Required columns check
    if "Trading Partner" not in df.columns:
        raise ValueError(f"'Trading Partner' column not found in {filename}")
    if "Exports ($M)" not in df.columns:
        raise ValueError(f"'Exports ($M)' column not found in {filename}")

    # --- Identify the correct RtW (%) column: the one immediately after Exports ($M) ---
    cols = list(df.columns)
    exp_index = cols.index("Exports ($M)")
    exports_rtw_col = cols[exp_index + 1]   # <-- THIS IS THE RtW (%) WE WANT

    # Build one row for this year
    row = {"Date": year}

    for out_col, partner_name in partner_map.items():
        partner_rows = df.loc[df["Trading Partner"] == partner_name, exports_rtw_col]

        if partner_rows.empty:
            row[out_col] = float("nan")
        else:
            row[out_col] = partner_rows.iloc[0]

    rows.append(row)

# Create final DataFrame
result = pd.DataFrame(rows)

# Sort by year
result = result.sort_values("Date").reset_index(drop=True)

# === EXPORT ===
output_excel = os.path.join(DATA_DIR, "UK_exports_RtW_panel.xlsx")
output_csv = os.path.join(DATA_DIR, "UK_exports_RtW_panel.csv")

# Uncomment if needed
result.to_excel(output_excel, index=False)
# result.to_csv(output_csv, index=False)

print(result)


    Date    EM    JP     EU     CH    AU     US
0   1998  0.76  2.28  58.17   1.96  1.56  13.91
1   1999  1.12  2.23  58.84   1.69  1.44  14.94
2   2000  1.26  2.32  57.56   1.69  1.47  15.38
3   2001  1.31  2.23  57.44   1.80  1.28  15.76
4   2002  1.24  2.00  58.84   1.57  1.25  15.49
5   2003  1.21  1.98  59.58   1.41  1.27  14.81
6   2004  1.40  1.96  59.70   1.39  1.33  14.06
7   2005  1.51  1.84  58.57   1.53  1.38  14.40
8   2006  1.65  1.69  59.83   1.41  1.26  13.87
9   2007  1.75  1.69  59.24   1.50  1.61  13.04
10  2008  2.00  1.56  57.59   1.44  1.84  12.50
11  2009  2.19  1.58  56.32   1.66  1.43  13.44
12  2010  2.81  1.59  55.71   1.74  1.41  12.60
13  2011  3.14  1.57  56.99   1.61  1.59  11.08
14  2012  3.63  1.57  52.32   3.05  1.59  11.87
15  2013  3.63  1.26  46.59  13.33  1.21  10.10
16  2014  4.89  1.33  50.18   7.02  1.21  11.29
17  2015  4.39  1.52  47.64   7.48  1.31  13.45
18  2016  4.73  1.65  50.60   4.82  1.43  13.77
19  2017  5.23  1.66  50.86   4.56  1.31